# Multimodal RAG with Image Generation

This notebook demonstrates a complete multimodal RAG (Retrieval-Augmented Generation) system that combines:

- **Image and Text Embeddings**: Using CLIP for images and multilingual E5 for text
- **Vector Database**: Qdrant for efficient similarity search
- **Content Generation**: Qwen2.5-VL for generating descriptions from retrieved examples
- **Advanced Retrieval**: MMR (Maximal Marginal Relevance) and deduplication

## Features
- Multi-modal search (text-to-image, image-to-image)
- Cross-modal reranking using CLIP
- MMR diversification to avoid redundant results
- Content generation inspired by retrieved examples
- Support for multilingual text (including Bangla)

## Table of Contents
1. [Setup and Installation](#setup)
2. [Configuration](#config)
3. [Data Loading and Utilities](#data)
4. [Embedding Models](#embeddings)
5. [Vector Database (Qdrant)](#qdrant)
6. [Data Ingestion](#ingestion)
7. [Search and Retrieval](#search)
8. [Content Generation](#generation)
9. [Complete Workflow Example](#workflow)
10. [Advanced Features](#advanced)


## 1. Setup and Installation {#setup}

First, let's install the required dependencies and import the necessary libraries.

**Required packages:**
- `sentence-transformers`: For embedding models (CLIP, E5)
- `qdrant-client`: Vector database client
- `transformers`: For the Qwen2.5-VL generation model
- `torch`: PyTorch for deep learning
- `pillow`: Image processing
- `tqdm`: Progress bars
- `numpy`: Numerical operations


In [1]:
# Install required packages (run this cell if packages are not installed)
import sys
!{sys.executable} -m pip install -U sentence-transformers qdrant-client pandas torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# Core imports
import os
import uuid
import json
import math
from typing import List, Dict, Any
import numpy as np
from PIL import Image
from tqdm import tqdm

# Embedding models
from sentence_transformers import SentenceTransformer

# Vector database
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct

# Generation model
from transformers import AutoProcessor
from transformers.models.qwen2_5_vl import Qwen2_5_VLForConditionalGeneration

print("All imports successful!")


Looking in indexes: https://download.pytorch.org/whl/cpu


/Users/sayems_mac/BrandReview/brandReviewModels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All imports successful!


## 2. Configuration {#config}

Define the configuration parameters for our multimodal RAG system.

**Key Components:**
- **CLIP ViT-L/14**: For image embeddings and cross-modal text-image understanding
- **Multilingual E5**: For text embeddings with support for multiple languages including Bangla
- **Qdrant**: In-memory vector database for this demo (can be configured for server mode)
- **Qwen2.5-VL-7B**: Vision-language model for content generation


In [2]:
# Embedding Models
IMG_EMB = "sentence-transformers/clip-ViT-L-14"  # CLIP for cross-modal embeddings
TXT_EMB = "intfloat/multilingual-e5-base"        # Multilingual text embeddings (Bangla-friendly)

# Vector Database
COLLECTION = "mm_posts"  # Collection name in Qdrant
USE_SERVER = False       # Use in-memory Qdrant for demo
QHOST, QPORT = "localhost", 6333  # Qdrant server settings

# Retrieval Parameters
TOPK_INIT = 24          # Initial number of candidates
TOPK_FINAL = 6          # Final number after reranking
MMR_LAMBDA = 0.7        # MMR diversification factor (0=relevance, 1=diversity)
DEDUP_THR = 0.96        # Deduplication threshold (cosine similarity)

# Generation Model
GEN_MODEL_ID = "Qwen/Qwen2.5-VL-3B-Instruct"

print("Configuration loaded successfully!")


Configuration loaded successfully!


## 3. Data Loading and Utilities {#data}

This section contains utility functions for data loading, normalization, and basic operations.

**Key Functions:**
- `load_json_any()`: Flexible JSON/JSONL loading
- `norm()`: Vector normalization for cosine similarity
- Error handling for file operations


In [3]:
def load_json_any(path: str) -> List[Dict[str, Any]]:
    """
    Load JSON data from file, supporting both JSON array and JSONL formats.
    
    Args:
        path: Path to the JSON/JSONL file
        
    Returns:
        List of dictionaries containing the loaded data
        
    Example:
        # JSON format: [{"image": "img1.jpg", "caption": "..."}, ...]
        # JSONL format: {"image": "img1.jpg", "caption": "..."}\n{"image": "img2.jpg", ...}
    """
    try:
        with open(path, "r", encoding="utf-8") as f:
            txt = f.read().strip()
        
        if txt.startswith("["):
            # Standard JSON array format
            return json.loads(txt)
        else:
            # JSONL format (one JSON object per line)
            return [json.loads(line) for line in txt.splitlines() if line.strip()]
    except Exception as e:
        print(f"Error loading {path}: {e}")
        return []

def norm(v: np.ndarray) -> np.ndarray:
    """
    Normalize vectors for cosine similarity computation.
    
    Args:
        v: Input vectors of shape (n_samples, n_features)
        
    Returns:
        Normalized vectors (L2 norm = 1)
        
    Note:
        Handles zero vectors by setting them to unit vectors
    """
    n = np.linalg.norm(v, axis=1, keepdims=True)
    n[n == 0] = 1  
    return v / n

# Test the utility functions
print("Utility functions defined successfully!")


posts = load_json_any("/Users/sayems_mac/BrandReview/brandReviewModels/RAG_pipeline/posts_data.json")
print(f"Loaded {len(posts)} posts")

# print some posts as a table
import pandas as pd

# Convert posts to DataFrame
df = pd.DataFrame(posts)

# Display the DataFrame
print(df.head())


Utility functions defined successfully!
Loaded 120 posts
            image              date                       caption  \
0  2025-09-23.jpg        2025-09-23             Us on a rainy day   
1  2025-09-22.jpg  2025-09-22 08:17  You got the order right Sir!   
2                  2025-09-20 21:12  Ekhon Texas eo DeliveryHobe.   
3                  2025-09-20 08:04       Esob kore ki moja PAAN?   
4                  2025-09-19 06:19                   Bucchen to?   

                                         description  \
0  Candid photo of an older man crouching beside ...   
1  Black‑and‑white portrait of a man (Rahat) with...   
2  X‑ray style graphic showing a human pelvis wit...   
3  Flat‑lay photograph of betel leaves arranged o...   
4  Bright red graphic featuring a boiled egg and ...   

                                          engagement  
0     {'reactions': 167, 'comments': 3, 'shares': 6}  
1  {'reactions': 3400, 'comments': 21, 'shares': ...  
2     {'reactions': 182, '

## 4. Embedding Models {#embeddings}

This section implements the embedding models for both images and text.

**Models Used:**
- **CLIP ViT-L/14**: For image embeddings and cross-modal text-image understanding
- **Multilingual E5**: For text embeddings with excellent multilingual support

**Key Features:**
- Lazy loading (models loaded only when needed)
- Automatic normalization for cosine similarity
- Support for both query and passage text encoding (E5 requirement)


In [4]:
# Global variables for model caching (lazy loading)
IMG = None
TXT = None

def get_img_encoder():
    """
    Get the image encoder (CLIP ViT-L/14).
    Uses global caching to avoid reloading the model.
    
    Returns:
        SentenceTransformer model for image embeddings
    """
    global IMG
    if IMG is None:
        print("Loading CLIP ViT-L/14 model...")
        IMG = SentenceTransformer(IMG_EMB)
        print("Image encoder loaded successfully!")
    return IMG

def get_txt_encoder():
    """
    Get the text encoder (Multilingual E5).
    Uses global caching to avoid reloading the model.
    
    Returns:
        SentenceTransformer model for text embeddings
    """
    global TXT
    if TXT is None:
        print("Loading Multilingual E5 model...")
        TXT = SentenceTransformer(TXT_EMB)
        print("Text encoder loaded successfully!")
    return TXT

def emb_image(paths: List[str]) -> np.ndarray:
    """
    Generate embeddings for a list of images.
    
    Args:
        paths: List of image file paths
        
    Returns:
        Normalized image embeddings of shape (n_images, embedding_dim)
        
    Note:
        Images are automatically converted to RGB format
    """
    enc = get_img_encoder()
    imgs = [Image.open(p).convert("RGB") for p in paths]
    return enc.encode(imgs, convert_to_numpy=True, normalize_embeddings=True).astype("float32")

def emb_text(texts: List[str], mode: str) -> np.ndarray:
    """
    Generate embeddings for text with proper E5 prefixing.
    
    Args:
        texts: List of text strings to embed
        mode: Either "query" or "passage" (affects E5 prefixing)
        
    Returns:
        Normalized text embeddings of shape (n_texts, embedding_dim)
        
    Note:
        E5 model requires specific prefixes: "query: " for queries, "passage: " for passages
    """
    prefix = "query: " if mode == "query" else "passage: "
    enc = get_txt_encoder()
    prefixed_texts = [prefix + t for t in texts]
    return enc.encode(prefixed_texts, convert_to_numpy=True, normalize_embeddings=True).astype("float32")

print("Embedding functions defined successfully!")


Embedding functions defined successfully!


## 5. Vector Database (Qdrant) {#qdrant}

This section implements the vector database operations using Qdrant.

**Qdrant Features:**
- **Multi-vector support**: Separate vectors for images and text
- **Cosine similarity**: Optimized for normalized embeddings
- **In-memory mode**: For demo purposes (can be configured for server mode)
- **Collection management**: Automatic recreation and cleanup

**Collection Structure:**
- **image**: CLIP embeddings (768 dimensions)
- **text**: E5 embeddings (768 dimensions)
- **payload**: Metadata (image path, caption, description, etc.)


In [5]:
# Global Qdrant client (lazy loading)
_Q = None

def qdrant() -> QdrantClient:
    """
    Get Qdrant client instance.
    Uses global caching and supports both in-memory and server modes.
    
    Returns:
        QdrantClient instance
    """
    global _Q
    if _Q is None:
        if USE_SERVER:
            print(f"Connecting to Qdrant server at {QHOST}:{QPORT}")
            _Q = QdrantClient(host=QHOST, port=QPORT)
        else:
            print("Using in-memory Qdrant database")
            _Q = QdrantClient(":memory:")
    return _Q

def recreate_collection(dim_img: int, dim_txt: int):
    """
    Create or recreate the vector collection with specified dimensions.
    
    Args:
        dim_img: Dimension of image embeddings
        dim_txt: Dimension of text embeddings
        
    Note:
        This will delete any existing collection with the same name
    """
    qc = qdrant()
    
    # Check if collection exists and delete it
    try:
        existing_collections = [c.name for c in qc.get_collections().collections]
        if COLLECTION in existing_collections:
            print(f"Deleting existing collection '{COLLECTION}'")
            qc.delete_collection(COLLECTION)
    except Exception as e:
        print(f"Warning: Could not check existing collections: {e}")
    
    # Create new collection
    print(f"Creating collection '{COLLECTION}' with image dim={dim_img}, text dim={dim_txt}")
    qc.recreate_collection(
        collection_name=COLLECTION,
        vectors_config={
            "image": VectorParams(size=dim_img, distance=Distance.COSINE),
            "text": VectorParams(size=dim_txt, distance=Distance.COSINE),
        }
    )
    print(f"Collection '{COLLECTION}' created successfully!")

def upsert(points: List[PointStruct]):
    """
    Insert or update points in the collection.
    
    Args:
        points: List of PointStruct objects containing vectors and payloads
    """
    qdrant().upsert(collection_name=COLLECTION, points=points)
    print(f"Upserted {len(points)} points to collection '{COLLECTION}'")

print("Qdrant functions defined successfully!")


Qdrant functions defined successfully!


## 6. Data Ingestion {#ingestion}

This section implements the data ingestion pipeline that processes posts and creates embeddings.

**Ingestion Process:**
1. **Load Data**: Parse JSON/JSONL files containing posts
2. **Prepare Images**: Extract image paths and load images
3. **Generate Embeddings**: Create both image and text embeddings
4. **Create Collection**: Set up Qdrant collection with proper dimensions
5. **Store Data**: Insert embeddings with metadata into vector database

**Data Format Expected:**
```json
{
  "image": "filename.jpg",
  "caption": "Image caption",
  "description": "Detailed description",
  "additional_field": "any other metadata"
}
```


In [6]:
def ingest(posts_json: str, images_dir):
    """
    Complete ingestion pipeline for multimodal posts.
    
    Args:
        posts_json: Path to JSON/JSONL file containing posts
        images_dir: Directory containing the image files
        
    Process:
        1. Load and parse posts data
        2. Extract image paths and prepare text content
        3. Generate image embeddings using CLIP
        4. Generate text embeddings using E5
        5. Create Qdrant collection with proper dimensions
        6. Store all data as points in the vector database
    """
    print("=== Starting Data Ingestion ===")

    print("luffy",images_dir)
    
    # Step 1: Load posts data
    print(f"Loading posts from {posts_json}...")
    posts = load_json_any(posts_json)

    if not posts:
        print("No posts loaded. Exiting.")
        return
    
    print(f"Loaded {len(posts)} posts")
    
    # Step 2: Prepare data
    print("Preparing image paths and text content...")
    paths = images_dir
    caps = [p.get("caption", "") for p in posts]
    descs = [p.get("description", "") for p in posts]
    
    # Combine caption and description for text embedding
    texts = [f"caption: {c}\n\ndescription: {d}" for c, d in zip(caps, descs)]
    
    # Step 3: Generate embeddings
    print(f"Generating embeddings for {len(paths)} images...")
    v_img = emb_image(paths)
    
    print(f"Generating embeddings for {len(texts)} text descriptions...")
    v_txt = emb_text(texts, mode="passage")
    
    print(f"Image embeddings shape: {v_img.shape}")
    print(f"Text embeddings shape: {v_txt.shape}")
    
    # Step 4: Create collection
    recreate_collection(v_img.shape[1], v_txt.shape[1])
    
    # Step 5: Create points and store
    print("Creating vector database points...")
    points = []
    
    for vi, vt, p, c, d in zip(v_img, v_txt, posts, caps, descs):
        # Create payload with all original data
        payload = {
            "image": p["image"],
            "caption": c,
            "description": d,
            # Include any additional fields from the original post
            **{k: v for k, v in p.items() if k not in ["image", "caption", "description"]}
        }
        
        # Create point with both image and text vectors
        point = PointStruct(
            id=str(uuid.uuid4()),
            vector={"image": vi.tolist(), "text": vt.tolist()},
            payload=payload
        )
        points.append(point)
    
    # Store in vector database
    upsert(points)
    print(f"=== Ingestion Complete: {len(points)} items stored ===")

print("Ingestion function defined successfully!")


Ingestion function defined successfully!


## 7. Search and Retrieval {#search}

This section implements advanced search and retrieval capabilities with reranking and diversification.

**Search Types:**
- **Text-to-Image**: Find images similar to a text query
- **Image-to-Image**: Find images similar to a query image

**Advanced Features:**
- **Cross-modal Reranking**: Use CLIP for text-image similarity
- **MMR Diversification**: Avoid redundant results using Maximal Marginal Relevance
- **Deduplication**: Remove highly similar results
- **Multi-stage Retrieval**: Initial ANN search + reranking + diversification


In [7]:
def search_by_text(query: str, topk: int) -> List[Dict[str, Any]]:
    """
    Search for images using a text query.
    
    Args:
        query: Text query string
        topk: Number of results to return
        
    Returns:
        List of search results with scores and payloads
    """
    qv = emb_text([query], mode="query")[0]
    res = qdrant().search(
        collection_name=COLLECTION,
        query_vector=("text", qv.tolist()),
        limit=topk,
        with_payload=True
    )
    return [{"score": r.score, "payload": r.payload} for r in res]

def search_by_image(img_path: str, topk: int) -> List[Dict[str, Any]]:
    """
    Search for similar images using an image query.
    
    Args:
        img_path: Path to the query image
        topk: Number of results to return
        
    Returns:
        List of search results with scores and payloads
    """
    qv = emb_image([img_path])[0]
    res = qdrant().search(
        collection_name=COLLECTION,
        query_vector=("image", qv.tolist()),
        limit=topk,
        with_payload=True
    )
    return [{"score": r.score, "payload": r.payload} for r in res]

def cosine(a, b):
    """Compute cosine similarity between two vectors."""
    return float(np.dot(a, b))

def mmr_diversify(vecs: np.ndarray, base_scores: np.ndarray, k: int, lam: float) -> List[int]:
    """
    Apply Maximal Marginal Relevance (MMR) for result diversification.
    
    Args:
        vecs: Normalized vectors (n_samples, n_features)
        base_scores: Relevance scores (n_samples,)
        k: Number of diverse results to select
        lam: Diversification factor (0=relevance only, 1=diversity only)
        
    Returns:
        List of indices for diverse results
        
    Algorithm:
        MMR = λ * Relevance - (1-λ) * max_similarity_to_selected
    """
    chosen = []
    candidates = list(range(len(vecs)))
    
    while candidates and len(chosen) < k:
        if not chosen:
            # Select the most relevant item first
            i = int(np.argmax(base_scores[candidates]))
            chosen.append(candidates.pop(candidates.index(i)))
            continue
        
        # Compute MMR scores for remaining candidates
        selected_vecs = np.stack([vecs[i] for i in chosen], 0)  # (m, d)
        similarities = vecs[candidates] @ selected_vecs.T  # (c, m)
        
        # MMR formula
        mmr_scores = lam * base_scores[candidates] - (1 - lam) * np.max(similarities, axis=1)
        
        # Select the item with highest MMR score
        i = candidates[int(np.argmax(mmr_scores))]
        candidates.remove(i)
        chosen.append(i)
    
    return chosen

def dedup_keep(vecs: np.ndarray, threshold: float) -> List[int]:
    """
    Remove duplicate vectors based on cosine similarity threshold.
    
    Args:
        vecs: Normalized vectors (n_samples, n_features)
        threshold: Similarity threshold for deduplication
        
    Returns:
        List of indices to keep (removing duplicates)
    """
    keep = []
    for i, v in enumerate(vecs):
        # Check if this vector is too similar to any already kept vector
        is_duplicate = any(float(v @ vecs[j]) >= threshold for j in keep)
        if not is_duplicate:
            keep.append(i)
    return keep

print("Search and retrieval functions defined successfully!")


Search and retrieval functions defined successfully!


In [8]:
def post_process_for_text_query(query: str, hits: List[Dict[str, Any]], images_dir) -> List[Dict[str, Any]]:
    """
    Advanced post-processing for text queries with cross-modal reranking, MMR, and deduplication.
    
    Args:
        query: Original text query
        hits: Initial search results from text vector search
        images_dir: Directory containing images (required for image path resolution)
        
    Returns:
        Reranked and diversified results
        
    Process:
        1. Extract image paths from hits
        2. Generate CLIP embeddings for images
        3. Rerank using CLIP text-image similarity
        4. Apply MMR diversification
        5. Remove duplicates
    """
    if not hits:
        return []
    
    if images_dir is None:
        print("Warning: images_dir not provided, using original hits without reranking")
        return hits
    
    print(f"Post-processing {len(hits)} hits with cross-modal reranking...")
    
    # Step 1: Extract image paths and generate CLIP embeddings
    img_paths = [h["payload"]["image"] for h in hits]
    full_img_paths = images_dir
    img_vecs = emb_image(full_img_paths)
    
    # Step 2: Cross-modal reranking using CLIP
    # Use CLIP's text encoder to get query embedding
    clip_txt = SentenceTransformer(IMG_EMB)  # CLIP text tower
    q_clip = clip_txt.encode([query], convert_to_numpy=True, normalize_embeddings=True).astype("float32")[0]
    
    # Compute text-image similarities
    relevance_scores = img_vecs @ q_clip  # cosine similarity (both normalized)
    
    # Step 3: Apply MMR diversification
    final_k = min(TOPK_FINAL, len(hits))
    diverse_indices = mmr_diversify(img_vecs, relevance_scores, final_k, MMR_LAMBDA)
    
    # Step 4: Deduplication
    selected_vecs = img_vecs[diverse_indices]
    keep_indices = dedup_keep(selected_vecs, DEDUP_THR)
    
    # Map back to original hit indices
    final_indices = [diverse_indices[i] for i in keep_indices]
    
    print(f"Selected {len(final_indices)} diverse, non-duplicate results")
    return [hits[i] for i in final_indices]

print("Advanced post-processing function defined successfully!")


Advanced post-processing function defined successfully!


## 8. Content Generation {#generation}

This section implements content generation using the Qwen2.5-VL vision-language model.

**Generation Features:**
- **Retrieval-Augmented**: Uses retrieved examples as context
- **Vision-Language Model**: Qwen2.5-VL for understanding both images and text
- **Style Consistency**: Generates content that matches the style of examples
- **Brand-Safe Content**: Configured for appropriate social media content

**Generation Process:**
1. **Context Preparation**: Format retrieved examples as context
2. **Model Loading**: Load Qwen2.5-VL with appropriate settings
3. **Prompt Construction**: Create structured prompt with examples and new image
4. **Content Generation**: Generate description following the established style


In [9]:
# Import torch for the generation function
import torch

# Generation style prompt
STYLE_PROMPT = (
    "You are a social content writer. Write ONE vivid sentence (15–30 words), "
    "brand-safe, lightly witty, grounded in the new image/caption and inspired by prior examples. "
    "No hashtags/emojis; do not invent facts beyond what's visible or stated."
)

# def gen_with_qwen_vl(new_image_path: str, new_caption: str, examples: List[Dict[str, Any]]) -> str:
#     """
#     Generate content using Qwen2.5-VL with retrieved examples as context.
    
#     Args:
#         new_image_path: Path to the new image for content generation
#         new_caption: Caption for the new image
#         examples: Retrieved examples to use as style reference
        
#     Returns:
#         Generated description text
        
#     Process:
#         1. Load Qwen2.5-VL model and processor
#         2. Format examples as context
#         3. Create multimodal prompt with image and text
#         4. Generate content following the style
#     """
#     print("Loading Qwen2.5-VL model...")
    
#     # Load model and processor
#     processor = AutoProcessor.from_pretrained(GEN_MODEL_ID, trust_remote_code=True)
#     model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
#         GEN_MODEL_ID, 
#         trust_remote_code=True, 
#         torch_dtype="auto", 
#         device_map="auto"
#     )
    
#     # Format examples as context
#     ex_lines = []
#     for i, e in enumerate(examples, 1):
#         caption = e['payload'].get('caption', '')
#         description = e['payload'].get('description', '')[:200]  # Truncate for brevity
#         ex_lines.append(f"[{i}] CAPTION: {caption}")
#         ex_lines.append(f"    DESCRIPTION: {description}")
    
#     # Create multimodal messages
#     messages = [
#         {"role": "system", "content": STYLE_PROMPT},
#         {"role": "user", "content": [
#             {"type": "text", "text": "EXAMPLES:\n" + "\n".join(ex_lines) + 
#                      f"\n\nNEW CAPTION: {new_caption}\nWrite the description for this image:"},
#             {"type": "image", "image": Image.open(new_image_path).convert("RGB")}
#         ]}
#     ]
    
#     # Generate content
#     print("Generating content...")
#     inputs = processor.apply_chat_template(
#         messages, 
#         add_generation_prompt=True, 
#         return_tensors="pt"
#     )

#     # Check if inputs is a string (tokenized) or dict (raw)
#     if isinstance(inputs, str):
#             # If it's a string, we need to tokenize it properly
#             inputs = processor.tokenizer(
#                 inputs, 
#                 return_tensors="pt", 
#                 padding=True, 
#                 truncation=True
#             )

#     # Move to device
#     inputs = {k: v.to(model.device) if hasattr(v, 'to') else v for k, v in inputs.items()}
    
#     with torch.no_grad():
#         outputs = model.generate(
#             **inputs, 
#             max_new_tokens=96, 
#             temperature=0.7, 
#             top_p=0.9, 
#             do_sample=True
#         )
    
#     # Decode the generated text
#     generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
    
#     # Extract only the generated part (remove the prompt)
#     if "Write the description for this image:" in generated_text:
#         generated_text = generated_text.split("Write the description for this image:")[-1].strip()
    
#     return generated_text

# def gen_with_qwen_vl(new_image_path:str, new_caption:str, examples:List[Dict[str,Any]])->str:
#     model_id = "Qwen/Qwen2.5-3B-Instruct"
#     from transformers import AutoTokenizer, AutoModelForCausalLM
#     processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
#     model = AutoModelForCausalLM.from_pretrained(
#         model_id,
#         trust_remote_code=True,
#         torch_dtype="auto",
#         device_map="auto",   # GPU/MPS if available, else CPU
#     )

#     ex_lines = []
#     for i,e in enumerate(examples,1):
#         ex_lines.append(f"[{i}] CAPTION: {e['payload'].get('caption','')}")
#         ex_lines.append(f"    DESCRIPTION: {e['payload'].get('description','')[:200]}")

#     messages = [
#         {"role":"system","content":STYLE_PROMPT},
#         {"role":"user","content":[
#             {"type":"text","text":"EXAMPLES:\n"+ "\n".join(ex_lines) + f"\n\nNEW CAPTION: {new_caption}\nWrite the description for this image:"},
#             {"type":"image","image":Image.open(new_image_path).convert("RGB")}
#         ]}
#     ]

#     print(messages)
    
#     inputs = processor.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
#     out = model.generate(**inputs, max_new_tokens=96, temperature=0.7, top_p=0.9, do_sample=True)
#     text = processor.batch_decode(out, skip_special_tokens=True)[0].strip()
#     return text

from typing import List, Dict, Any
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq

def gen_with_qwen_vl(new_image_path: str, new_caption: str, examples: List[Dict[str, Any]]) -> str:
    model_id = "Qwen/Qwen2.5-VL-3B-Instruct"  # or 7B

    processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
    model = AutoModelForVision2Seq.from_pretrained(
        model_id,
        trust_remote_code=True,
        torch_dtype="auto",
        device_map="auto",
    )

    ex_lines = []
    for i, e in enumerate(examples, 1):
        cap = e.get("payload", {}).get("caption", "")
        desc = e.get("payload", {}).get("description", "")[:200]
        ex_lines.append(f"[{i}] CAPTION: {cap}")
        ex_lines.append(f"    DESCRIPTION: {desc}")
    example_text = "EXAMPLES:\n" + "\n".join(ex_lines)

    img = Image.open(new_image_path).convert("RGB")

    messages = [
        {"role": "system", "content": STYLE_PROMPT},
        {"role": "user", "content": [
            {"type": "text", "text": f"{example_text}\n\nNEW CAPTION: {new_caption}\nWrite the description for this image:"},
            {"type": "image", "image": img},
        ]},
    ]

    # 1) Build prompt text only (do NOT request tensors here)
    prompt_text = processor.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,          # ensure we get a string
    )

    # 2) Convert to tensors with text+image together
    inputs = processor(
        text=[prompt_text],
        images=[img],
        return_tensors="pt",
        padding=True,
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # 3) Generate
    out = model.generate(
        **inputs,
        max_new_tokens=96,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        eos_token_id=processor.tokenizer.eos_token_id,
        pad_token_id=processor.tokenizer.pad_token_id,
    )

    return processor.batch_decode(out, skip_special_tokens=True)[0].strip()



print("Content generation function defined successfully!")


Content generation function defined successfully!


## 9. Complete Workflow Example {#workflow}

This section demonstrates the complete multimodal RAG workflow from data ingestion to content generation.

**Workflow Steps:**
1. **Data Ingestion**: Load and embed posts data
2. **Text Search**: Find similar content using text query
3. **Content Generation**: Generate new content using retrieved examples
4. **Image Search**: Find similar images using image query
5. **Results Display**: Show search results and generated content


In [10]:


def run_complete_workflow(posts_json: str, images_dir: str, 
                         query_text: str = None, query_image: str = None,
                         generate_image: str = None, generate_caption: str = ""):
    """
    Run the complete multimodal RAG workflow.
    
    Args:
        posts_json: Path to posts data file
        images_dir: Directory containing images
        query_text: Text query for search (optional)
        query_image: Image path for search (optional)
        generate_image: Image path for content generation (optional)
        generate_caption: Caption for content generation (optional)
    """
    print("🚀 Starting Complete Multimodal RAG Workflow")
    print("=" * 60)
    
    # Step 1: Data Ingestion
    print("\n📥 Step 1: Data Ingestion")
    ingest(posts_json, images_dir)
    
    hits = []
    
    # Step 2: Search (Text or Image)
    if query_text:
        print(f"\n🔍 Step 2: Text Search for '{query_text}'")
        # Initial text search
        hits = search_by_text(query_text, TOPK_INIT)
        print(f"Found {len(hits)} initial results")
        
        # Advanced post-processing
        hits = post_process_for_text_query(query_text, hits, images_dir)
        print(f"After reranking and diversification: {len(hits)} results")
        
    elif query_image:
        print(f"\n🔍 Step 2: Image Search for '{query_image}'")
        hits = search_by_image(query_image, TOPK_INIT)
        print(f"Found {len(hits)} similar images")
    
    # Step 3: Display Results
    if hits:
        print(f"\n📋 Step 3: Search Results")
        print("-" * 40)
        for i, h in enumerate(hits, 1):
            payload = h['payload']
            print(f"[{i}] Score: {h['score']:.3f}")
            print(f"    Image: {payload.get('image', 'N/A')}")
            print(f"    Caption: {payload.get('caption', 'N/A')[:80]}...")
            print(f"    Description: {payload.get('description', 'N/A')[:80]}...")
            print()
    
    # Step 4: Content Generation
    if generate_image and hits:
        print(f"\n✨ Step 4: Content Generation")
        print(f"Generating content for image: {generate_image}")
        print(f"Caption: {generate_caption}")
        print(f"Using {len(hits[:3])} examples as style reference...")
        
        try:
            generated_desc = gen_with_qwen_vl(generate_image, generate_caption, hits[:3])
            print(f"\n🎯 Generated Description:")
            print(f"'{generated_desc}'")
        except Exception as e:
            print(f"❌ Generation failed: {e}")
    
    print("\n✅ Workflow Complete!")
    print("=" * 60)

print("Complete workflow function defined successfully!")


Complete workflow function defined successfully!


## 10. Example Usage {#examples}

Now let's run some examples to demonstrate the multimodal RAG system in action.

**Note**: Make sure you have the data files (`posts_data.json` and images directory) in the correct location before running these examples.


In [11]:
# Example 1: Complete workflow with text search and content generation
# Uncomment and modify the paths below to run this example


# Set your data paths
POSTS_FILE = "../posts_data.json"  # Path to your posts data
IMAGES_DIR = "/Users/sayems_mac/BrandReview/brandReviewModels/RAG_pipeline/images"           # Path to your images directory

# Example text query
TEXT_QUERY = "rainy day scene"

# Example content generation
GENERATE_IMAGE = "../images/2025-09-23.jpg"  # Path to image for generation
GENERATE_CAPTION = "oi mama !"


from pathlib import Path

def normalize_image_inputs(images_dir_or_list, recursive=True):
    exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp"}
    def collect_dir(d: Path):
        it = d.rglob("*") if recursive else d.glob("*")
        return [str(fp) for fp in it if fp.is_file() and fp.suffix.lower() in exts]

    if isinstance(images_dir_or_list, (list, tuple)):
        files = []
        for item in images_dir_or_list:
            p = Path(item)
            if p.is_dir():
                files.extend(collect_dir(p))
            elif p.is_file() and p.suffix.lower() in exts:
                files.append(str(p))
        return files

    p = Path(images_dir_or_list)
    if p.is_dir():
        return collect_dir(p)
    if p.is_file():
        return [str(p)]
    raise FileNotFoundError(f"Path not found: {p}")

# Update your call:
image_files = normalize_image_inputs(IMAGES_DIR) 

# Run the complete workflow

run_complete_workflow(
    posts_json=POSTS_FILE,
    images_dir=image_files,
    query_text=TEXT_QUERY,
    generate_image=GENERATE_IMAGE,
    generate_caption=GENERATE_CAPTION
)


🚀 Starting Complete Multimodal RAG Workflow

📥 Step 1: Data Ingestion
=== Starting Data Ingestion ===
luffy ['/Users/sayems_mac/BrandReview/brandReviewModels/RAG_pipeline/images/2025-09-23.jpg', '/Users/sayems_mac/BrandReview/brandReviewModels/RAG_pipeline/images/2025-09-22.jpg']
Loading posts from ../posts_data.json...
Loaded 120 posts
Preparing image paths and text content...
Generating embeddings for 2 images...
Loading CLIP ViT-L/14 model...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Image encoder loaded successfully!
Generating embeddings for 120 text descriptions...
Loading Multilingual E5 model...
Text encoder loaded successfully!
Image embeddings shape: (2, 768)
Text embeddings shape: (120, 768)
Using in-memory Qdrant database
Creating collection 'mm_posts' with image dim=768, text dim=768
Collection 'mm_posts' created successfully!
Creating vector database points...
Upserted 2 points to collection 'mm_posts'
=== Ingestion Complete: 2 items stored ===

🔍 Step 2: Text Search for 'rainy day scene'
Found 2 initial results
Post-processing 2 hits with cross-modal reranking...


/var/folders/mb/qsr33_nx15bbzcr3kfv4g67h0000gn/T/ipykernel_9034/1523765075.py:46: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qc.recreate_collection(
/var/folders/mb/qsr33_nx15bbzcr3kfv4g67h0000gn/T/ipykernel_9034/1223577332.py:13: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  res = qdrant().search(


Selected 2 diverse, non-duplicate results
After reranking and diversification: 2 results

📋 Step 3: Search Results
----------------------------------------
[1] Score: 0.845
    Image: 2025-09-23.jpg
    Caption: Us on a rainy day...
    Description: Candid photo of an older man crouching beside a storm‑drain feeding puffed rice ...

[2] Score: 0.756
    Image: 2025-09-22.jpg
    Caption: You got the order right Sir!...
    Description: Black‑and‑white portrait of a man (Rahat) with a speech bubble: ‘Ordered boneles...


✨ Step 4: Content Generation
Generating content for image: ../images/2025-09-23.jpg
Caption: oi mama !
Using 2 examples as style reference...


The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
/Users/sayems_mac/BrandReview/brandReviewModels/.venv/lib/python3.11/site-packages/transformers/models/auto/modeling_auto.py:2242: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.27s/it]
Some parameters are on the meta device because they were offloaded to the disk.



🎯 Generated Description:
'system
You are a social content writer. Write ONE vivid sentence (15–30 words), brand-safe, lightly witty, grounded in the new image/caption and inspired by prior examples. No hashtags/emojis; do not invent facts beyond what's visible or stated.
user
EXAMPLES:
[1] CAPTION: Us on a rainy day
    DESCRIPTION: Candid photo of an older man crouching beside a storm‑drain feeding puffed rice to a rat emerging from the grate; Delivery Hobe! logo is in the corner.
[2] CAPTION: You got the order right Sir!
    DESCRIPTION: Black‑and‑white portrait of a man (Rahat) with a speech bubble: ‘Ordered boneless chicken, they delivered Eggcellent service.’  The words ‘chicken’ and ‘egg’ are highlighted in red and a row of golden

NEW CAPTION: oi mama !
Write the description for this image:
assistant
A man feeds a rat a piece of bread while crouched next to a storm drain, with a colorful mural and a phone number in the corner.'

✅ Workflow Complete!


In [12]:
# Example 2: Individual function usage
# Uncomment sections below to test individual components

"""
# Test data loading
print("=== Testing Data Loading ===")
posts = load_json_any("../posts_data.json")
print(f"Loaded {len(posts)} posts")
if posts:
    print(f"Sample post keys: {list(posts[0].keys())}")
    print(f"Sample post: {posts[0]}")

# Test embedding functions
print("\n=== Testing Embeddings ===")
if posts:
    # Test text embedding
    sample_text = posts[0].get('caption', '') + ' ' + posts[0].get('description', '')
    text_emb = emb_text([sample_text], mode="passage")
    print(f"Text embedding shape: {text_emb.shape}")
    
    # Test image embedding (if image exists)
    sample_image = f"../images/{posts[0]['image']}"
    if os.path.exists(sample_image):
        img_emb = emb_image([sample_image])
        print(f"Image embedding shape: {img_emb.shape}")

# Test search functions (after ingestion)
print("\n=== Testing Search ===")
# First run ingestion
ingest("../posts_data.json", "../images")

# Test text search
results = search_by_text("rainy day", 5)
print(f"Text search returned {len(results)} results")

# Test image search (if image exists)
if posts and os.path.exists(f"../images/{posts[0]['image']}"):
    img_results = search_by_image(f"../images/{posts[0]['image']}", 5)
    print(f"Image search returned {len(img_results)} results")
"""

print("Individual function test code ready!")


Individual function test code ready!


## 11. Advanced Features {#advanced}

This section covers advanced features and customization options.

### 11.1 Configuration Tuning

**Retrieval Parameters:**
- `TOPK_INIT`: Number of initial candidates (higher = more recall, slower)
- `TOPK_FINAL`: Final number of results (lower = more focused)
- `MMR_LAMBDA`: Diversification vs relevance balance (0.7 = good default)
- `DEDUP_THR`: Deduplication threshold (0.96 = strict, 0.9 = lenient)

**Model Selection:**
- Image embeddings: CLIP variants (ViT-B/32, ViT-L/14, ViT-H/14)
- Text embeddings: E5 variants (small, base, large) or other multilingual models
- Generation: Qwen2.5-VL variants (3B, 7B, 14B, 72B)

### 11.2 Performance Optimization

**For Large Datasets:**
- Use Qdrant server mode for persistence
- Batch embedding generation
- Implement embedding caching
- Use GPU acceleration for models

**For Production:**
- Implement async processing
- Add error handling and retries
- Monitor embedding quality
- Cache generated content


In [13]:
# Advanced configuration examples
# Uncomment and modify as needed

"""
# Performance tuning examples
PERFORMANCE_CONFIG = {
    # For faster processing (lower quality)
    "fast": {
        "TOPK_INIT": 12,
        "TOPK_FINAL": 3,
        "MMR_LAMBDA": 0.5,
        "DEDUP_THR": 0.9,
        "IMG_EMB": "sentence-transformers/clip-ViT-B-32",
        "GEN_MODEL_ID": "Qwen/Qwen2.5-VL-3B-Instruct"
    },
    
    # For high quality (slower processing)
    "quality": {
        "TOPK_INIT": 50,
        "TOPK_FINAL": 10,
        "MMR_LAMBDA": 0.8,
        "DEDUP_THR": 0.98,
        "IMG_EMB": "sentence-transformers/clip-ViT-H-14",
        "GEN_MODEL_ID": "Qwen/Qwen2.5-VL-72B-Instruct"
    }
}

# Apply configuration
def apply_config(config_name):
    config = PERFORMANCE_CONFIG[config_name]
    globals().update(config)
    print(f"Applied {config_name} configuration")

# Example usage:
# apply_config("fast")  # For quick testing
# apply_config("quality")  # For production quality
"""

print("Advanced configuration examples ready!")


Advanced configuration examples ready!


## 12. Troubleshooting and Tips {#troubleshooting}

### Common Issues and Solutions

**Memory Issues:**
- Use smaller models for development (CLIP ViT-B/32, Qwen2.5-VL-3B)
- Process data in batches
- Use `torch.no_grad()` for inference

**Slow Performance:**
- Enable GPU acceleration
- Use in-memory Qdrant for small datasets
- Cache embeddings to avoid recomputation
- Reduce `TOPK_INIT` and `TOPK_FINAL`

**Poor Search Results:**
- Check embedding quality with sample queries
- Adjust MMR_LAMBDA for better diversity/relevance balance
- Verify text preprocessing (E5 prefixing)
- Ensure image paths are correct

**Generation Issues:**
- Verify model compatibility with your hardware
- Check image format (should be RGB)
- Ensure sufficient GPU memory for generation model
- Try different temperature and top_p values

### Data Format Requirements

**Posts JSON Format:**
```json
[
  {
    "image": "filename.jpg",
    "caption": "Image caption",
    "description": "Detailed description",
    "date": "2025-01-01",
    "engagement": {"likes": 100, "comments": 5}
  }
]
```

**Image Requirements:**
- Supported formats: JPG, PNG, WEBP
- Automatic RGB conversion
- Reasonable file sizes (< 10MB recommended)


## 13. Conclusion {#conclusion}

This notebook demonstrates a complete multimodal RAG system that combines:

### Key Features Implemented:
- ✅ **Multimodal Embeddings**: CLIP for images, E5 for text
- ✅ **Vector Database**: Qdrant with multi-vector support
- ✅ **Advanced Retrieval**: MMR diversification and deduplication
- ✅ **Cross-modal Reranking**: CLIP text-image similarity
- ✅ **Content Generation**: Qwen2.5-VL for style-consistent output
- ✅ **Multilingual Support**: E5 model supports multiple languages

### System Architecture:
```
Data → Embeddings → Vector DB → Search → Rerank → Generate
```

### Use Cases:
- **Content Creation**: Generate social media descriptions
- **Image Search**: Find similar images by text or image
- **Style Transfer**: Adapt content to match existing examples
- **Multilingual RAG**: Support for non-English queries

### Next Steps:
1. **Scale Up**: Use server-mode Qdrant for production
2. **Fine-tune**: Adapt models to your specific domain
3. **Monitor**: Add logging and performance metrics
4. **Deploy**: Create API endpoints for production use

### Resources:
- [Qdrant Documentation](https://qdrant.tech/documentation/)
- [Sentence Transformers](https://www.sbert.net/)
- [Qwen2.5-VL Model](https://huggingface.co/Qwen/Qwen2.5-VL-7B-Instruct)
- [CLIP Model](https://huggingface.co/sentence-transformers/clip-ViT-L-14)

---

**Happy Building! 🚀**

*This notebook provides a solid foundation for building sophisticated multimodal RAG applications. Feel free to modify and extend it for your specific needs.*
